In [24]:
# Import Required Libraries
import numpy as np
import time 
from sklearn.neighbors import NearestNeighbors

<h2>Step 1: Data Preparation</h2>

In [25]:
# Set random seed so we can reproduce the numbers
np.random.seed(1)
veclen = 10

# Create random sample of 100,000 rows based on a normal distribution
vecs = [np.random.normal(0, 1, veclen) for _ in range(100000)]

In [26]:
# Show the first 5 rows in the list
vecs[:5]

[array([ 1.62434536, -0.61175641, -0.52817175, -1.07296862,  0.86540763,
        -2.3015387 ,  1.74481176, -0.7612069 ,  0.3190391 , -0.24937038]),
 array([ 1.46210794, -2.06014071, -0.3224172 , -0.38405435,  1.13376944,
        -1.09989127, -0.17242821, -0.87785842,  0.04221375,  0.58281521]),
 array([-1.10061918,  1.14472371,  0.90159072,  0.50249434,  0.90085595,
        -0.68372786, -0.12289023, -0.93576943, -0.26788808,  0.53035547]),
 array([-0.69166075, -0.39675353, -0.6871727 , -0.84520564, -0.67124613,
        -0.0126646 , -1.11731035,  0.2344157 ,  1.65980218,  0.74204416]),
 array([-0.19183555, -0.88762896, -0.74715829,  1.6924546 ,  0.05080775,
        -0.63699565,  0.19091548,  2.10025514,  0.12015895,  0.61720311])]

In [27]:
# Set random seed so we can reproduce the numbers
np.random.seed(2)

# Generate a random sample
q = np.random.normal(0, 1, veclen)

In [28]:
# Normalize the values
maxnorm = max([np.linalg.norm(v) for v in vecs])

In [29]:
# Function to calculate the euclidean distance between the observations
def transform(vecs):
    maxnorm = max([np.linalg.norm(v) for v in vecs])
    new_vecs = []
    for v in vecs:
        new_vecs.append(np.insert(v, 0, np.sqrt(maxnorm**2-np.linalg.norm(v)**2)))
    return new_vecs

In [30]:
# Calculate the distance for all rows
vecs_trans = transform(vecs)

In [31]:
# Check the distance calculation and append
vecs_trans[1000]

array([ 6.32518175, -0.12247391,  0.22816982, -0.35230513, -0.83055344,
       -0.26108982,  0.16935423,  0.6736231 , -0.32720161, -0.30529915,
        0.52486533])

In [32]:
# Use the generated single random sample as a sample transaction to test
q_trans = np.insert(q, 0, 0)

In [33]:
# Display the transaction
q_trans

array([ 0.        , -0.41675785, -0.05626683, -2.1361961 ,  1.64027081,
       -1.79343559, -0.84174737,  0.50288142, -1.24528809, -1.05795222,
       -0.90900761])

In [34]:
# Use the dot product to calculate the compare the sample transaction to all the transactions seen so far
def mips_naive(q, vecs):
    mip = -1e10
    idx = -1
    for i, v in enumerate(vecs):
        if np.dot(q, v) > mip:
            mip = np.dot(q, v)
            idx = i
    return idx, mip

In [35]:
# Find the index of a seen transaction that is the closest to the comparison transaction and calculate time taken for the comparison process
start = time.time()
idx,_ = mips_naive(q, vecs)
print("Min index", idx)
print("Elapsed time", time.time()-start)

Min index 50753
Elapsed time 0.13890886306762695


In [36]:
X = np.array(vecs_trans)

<h2>Step 2: Perform K-NN algorithm</h2>

In [39]:
# Use KNN to fit the transactions to a KNN model
nbrs = NearestNeighbors(n_neighbors=1, algorithm='kd_tree').fit(X)

In [41]:
# Calculate the time taken to use KNN to compare the sample transaction and find the nearest transaction that was seen before
start = time.time()
distances, indices = nbrs.kneighbors(np.array([q_trans]))
print("Min index", indices[0])
print("Elapsed time", time.time()-start)

Min index [50753]
Elapsed time 0.0052530765533447266


In [42]:
indices

array([[50753]])

In [36]:
distances

array([[4.89652365]])

In [23]:
# Display the size of the total population
X.shape

(100000, 11)

In [24]:
# Get the corresponding features for the closest transaction
X[indices[0]]

array([[ 4.28969653, -0.5965094 ,  1.42071035, -3.37551897,  0.73652411,
        -1.97169694, -1.35898454,  0.64774945, -1.61868989, -0.50793098,
        -0.41328024]])

In [25]:
# Check the above features with the features from the sample transaction
q_trans

array([ 0.        , -0.41675785, -0.05626683, -2.1361961 ,  1.64027081,
       -1.79343559, -0.84174737,  0.50288142, -1.24528809, -1.05795222,
       -0.90900761])